In [27]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Get the drop down options for Task 1
locations = list(set(spacex_df['Launch Site'].values))
dropdownoptions = [{'label': 'ALL', 'value': 'ALL'}]
for location in locations:
    dropdownoptions.append( {'label': location, 'value': location} )
    
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options = dropdownoptions, value='ALL', placeholder='Select a launchsite to investigate', searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', max=max_payload, min=min_payload, value=[min_payload, max_payload], step=1000),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
)

def get_pie_chart( entered_site ):
    if entered_site == 'ALL':
        filtered_df = spacex_df[['Launch Site', 'class']].groupby('Launch Site').sum()
        filtered_df.reset_index(inplace=True)
        fig=px.pie(filtered_df, values='class', names='Launch Site')
    else:
        filtered_df = spacex_df.loc[ spacex_df['Launch Site'] == entered_site ]
        filtered_df = pd.DataFrame(filtered_df[['class']].value_counts())
        filtered_df.reset_index(inplace=True)
        filtered_df.columns = ['class', 'count']
        fig=px.pie(filtered_df, values='count', names='class')
    return fig
        
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [ Input(component_id='site-dropdown', component_property='value'), 
        Input(component_id="payload-slider", component_property="value") ]
)

# def get_scatter_plot( payloadval ):
def create_scatter(site, payload):
    if site == 'ALL':
        filter_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
        fig=px.scatter(filter_df,x='Payload Mass (kg)', y='class',color="Booster Version Category")

    else:
        filter_df = spacex_df[spacex_df['Launch Site']==select_dropdown]
        filter_df = filter_df[(filter_df['Payload Mass (kg)']>=int(payload[0])) & (filter_df['Payload Mass (kg)'] <=int(payload[1]))]
        print(filter_df[['class','Payload Mass (kg)']])
        fig=px.scatter(filter_df,x='Payload Mass (kg)', y='class',color="Booster Version Category")
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jul/2023 13:13:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 13:13:54] "POST /_dash-update-component HTTP/1.1" 200 -
